In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e20/sample_submission.csv
/kaggle/input/playground-series-s3e20/train.csv
/kaggle/input/playground-series-s3e20/test.csv


In [2]:
import numpy as np
import pandas as pd

train_df = pd.read_csv('/kaggle/input/playground-series-s3e20/train.csv')

submit_df = pd.read_csv('/kaggle/input/playground-series-s3e20/sample_submission.csv')
submit_df.emission = 0.992 * np.max([train_df[(train_df.year == y) & (train_df.week_no < 49)].emission for y in range(2019, 2022)], axis=0)
submit_df.to_csv('submission_1.csv', index=False)

In [3]:
train = pd.read_csv('/kaggle/input/playground-series-s3e20/train.csv', index_col="ID_LAT_LON_YEAR_WEEK")
test = pd.read_csv('/kaggle/input/playground-series-s3e20/test.csv', index_col="ID_LAT_LON_YEAR_WEEK")
submission = pd.read_csv('/kaggle/input/playground-series-s3e20/sample_submission.csv')

train['location'] = list(zip(train['latitude'], train['longitude']))
train = train[train.week_no <= 48]


def get_prev_max_emissions(location):    
    emi_2019 = train.loc[(train.year == 2019) & (train.location == location), 'emission'].values
    emi_2020 = train.loc[(train.year == 2020) & (train.location == location), 'emission'].values
    emi_2021 = train.loc[(train.year == 2021) & (train.location == location), 'emission'].values
    return np.max([emi_2019, emi_2020, emi_2021], axis=0)
    
predictions = [get_prev_max_emissions(loc) for loc in train.location.unique()]

test['emission'] = np.hstack(predictions)
test.loc[test['longitude'] == 29.321, 'emission'] = train.loc[(train['year'] == 2021) & 
                                                              (train['longitude'] == 29.321),
                                                              'emission'].values
submission['emission'] = test['emission'].to_list()
submission.to_csv('submission.csv', index=False)